In [2]:
from turbo import TurboM
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
# loading UMAP embeddings & labels
umap_df = pd.read_csv("./data/umap_fit.csv")
umap_df = umap_df.iloc[:, 1:3] # select only first 2 dimensions, can change for up to d dimensions
labels_df = pd.read_csv("./data/labels_sorted.csv")
labels = labels_df['activity'] # I prefer 'activity' over 'activity_scaled' because the latter is unobtainable during actual experimentation


# running through 10 iterations of local bayesian optimisation to get an average of results
for i in range(0, 10):

    # defining the objective function 
    class DataObjective:
        def __init__(self, data, labels):
            """
            Initializes the objective function with data and labels.
            Keeps track of which data points have already been used.
            """
            self.data = np.asarray(data)
            self.labels = np.asarray(labels)
            self.used = np.zeros(len(self.data), dtype=bool)
        
            # define lower and upper bounds of embeddings for optimisation
            self.lb = self.data.min(axis=0)
            self.ub = self.data.max(axis=0)
        
        def __call__(self, x):
            """
            Given a candidate point x, returns the label (activity) of the closest row in the dataset.
            Ensures that already used rows are not considered.
            """
            x = np.asarray(x).reshape(-1)
            if x.shape[0] != self.data.shape[1]:
                raise ValueError("Input dimension does not match the data dimension.")
            
            # calculate euclidean distances 
            #TODO: can potentially change distance metric to something else
            distances = np.linalg.norm(self.data - x, axis=1)

            # ignore rows that have already been used
            distances[self.used] = np.inf

            # if all points have been used, raise error
            if np.all(np.isinf(distances)):
                raise ValueError("All data used.")

            # find idnex of row with the minimum distance
            nearest_idx = np.argmin(distances)

            # mark this row as used
            self.used[nearest_idx] = True

            # retrieve the label (activity) of the selected row
            output = self.labels[nearest_idx]
            return -output # returning the negative value since TuRBO by default optimises for global minima


    f = DataObjective(umap_df, labels)

    # running bayesian optimisation
    outcome_list = [] # list to store the percentage of high-activity mutants

    turbo_m = TurboM(
        f=f,  # objective function
        lb=f.lb,  # Numpy array specifying lower bounds
        ub=f.ub,  # Numpy array specifying upper bounds
        n_init=16,  # Number of initial bounds from an Symmetric Latin hypercube design
        max_evals=160,  # Maximum number of evaluations
        n_trust_regions=2,  # Number of trust regions
        batch_size=8,  # How large batch size TuRBO uses
        verbose=False,  # Print information from each batch
        use_ard=True,  # Use ARD (Automatic Relevance Determination) for Gaussian Process kernel
        max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos decomposition
        n_training_steps=30,  # Number of steps of ADAM to learn the hypers
        min_cuda=1024,  # Run on the CPU for small datasets
        device="cpu",  # "cpu" or "cuda"
        dtype="float64",  # float64 or float32
        )
    
    turbo_m.optimize() # start optimisation process

    # retrieve the optimised data points and their corresponding function values
    X = turbo_m.X
    fX = turbo_m.fX
    selected_protein_df = pd.DataFrame(X)
    selected_protein_df['activity'] = -fX

    # calculating % of high activity mutants
    high_activity_proteins = selected_protein_df[selected_protein_df['activity']>=2.8] # for Jones data, anything with activity >2.8 is considered "high activity" (>50th percentile of activity)
    outcome_list.append(len(high_activity_proteins)/16)

print(f"average % of high activity mutants:{sum(outcome_list)/len(outcome_list)*100}%")

average % of high activity mutants:150.0%
